In [6]:
import time
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, regularizers
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PowerTransformer, QuantileTransformer, MaxAbsScaler
from sklearn.feature_selection import SelectKBest, mutual_info_classif
from sklearn.neighbors import LocalOutlierFactor
from imblearn.over_sampling import ADASYN
from sklearn.metrics import classification_report, confusion_matrix
import joblib
import matplotlib.pyplot as plt
import seaborn as sns
import multiprocessing as mp
from functools import partial
import os
import json
from scipy import stats

# Suppress TensorFlow logs for cleaner output
tf.get_logger().setLevel('ERROR')

# --- EnhancedAdaptiveNIDS: Layer 1 (Autoencoder for Anomaly Detection) ---
class EnhancedAdaptiveNIDS:
    def __init__(self, input_dim, latent_dim=32, learning_rate=1e-4):
        self.input_dim = input_dim
        self.latent_dim = latent_dim
        self.learning_rate = learning_rate
        self.model = self._build_autoencoder()
        self.threshold = None

    def _build_autoencoder(self):
        inputs = layers.Input(shape=(self.input_dim,))
        x = layers.BatchNormalization()(inputs)
        x = layers.Dense(128, activation='relu')(x)
        x = layers.BatchNormalization()(x)
        x = layers.Dropout(0.2)(x)
        x = layers.Dense(64, activation='relu')(x)
        x = layers.BatchNormalization()(x)
        x = layers.Dropout(0.2)(x)
        encoded = layers.Dense(self.latent_dim, activation='relu')(x)
        x = layers.Dense(64, activation='relu')(encoded)
        x = layers.BatchNormalization()(x)
        x = layers.Dropout(0.2)(x)
        x = layers.Dense(128, activation='relu')(x)
        x = layers.BatchNormalization()(x)
        x = layers.Dropout(0.2)(x)
        decoded = layers.Dense(self.input_dim, activation='linear')(x)
        autoencoder = keras.Model(inputs=inputs, outputs=decoded)
        autoencoder.compile(optimizer=keras.optimizers.Adam(learning_rate=self.learning_rate),
                            loss='mean_squared_error')
        return autoencoder

    def train(self, X_train, X_val, epochs=50, batch_size=64):
        early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
        reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6)
        history = self.model.fit(X_train, X_train, epochs=epochs, batch_size=batch_size,
                                 validation_data=(X_val, X_val), callbacks=[early_stopping, reduce_lr], verbose=1)
        plt.figure(figsize=(12, 4))
        plt.subplot(1, 2, 1)
        plt.plot(history.history['loss'], label='Training Loss')
        plt.plot(history.history['val_loss'], label='Validation Loss')
        plt.title('Autoencoder Loss')
        plt.legend()
        plt.savefig('autoencoder_loss.png')
        plt.close()
        self._set_dynamic_threshold(X_train)
        return history

    def _set_dynamic_threshold(self, X_data):
        reconstructed = self.model.predict(X_data)
        mse = np.mean(np.square(X_data - reconstructed), axis=1)
        self.threshold = np.percentile(mse, 95)
        print(f"Dynamic threshold set to: {self.threshold}")
        plt.figure(figsize=(10, 6))
        plt.hist(mse, bins=50)
        plt.axvline(self.threshold, color='r', linestyle='--', label=f'Threshold: {self.threshold:.6f}')
        plt.title('Reconstruction Error Distribution')
        plt.xlabel('Mean Squared Error')
        plt.ylabel('Frequency')
        plt.legend()
        plt.savefig('error_distribution.png')
        plt.close()

    def detect_anomalies(self, X_data):
        if self.threshold is None:
            raise ValueError("Model hasn't been trained yet. Call train() first.")
        reconstructed = self.model.predict(X_data)
        errors = np.mean(np.square(X_data - reconstructed), axis=1)
        anomaly_indices = np.where(errors > self.threshold)[0]
        confidence = errors / np.max(errors) if len(errors) > 0 else np.array([])
        return X_data[anomaly_indices], anomaly_indices, errors, confidence

    def get_encoded_features(self, X_data):
        encoder = keras.Model(inputs=self.model.input, outputs=self.model.layers[6].output)
        return encoder.predict(X_data)

# --- AdaptiveNIDSLayer2: Layer 2 (CNN-BiLSTM for Classification) ---
class AdaptiveNIDSLayer2:
    def __init__(self, input_dim, num_classes, seq_length=10):
        self.input_dim = input_dim
        self.num_classes = num_classes
        self.seq_length = seq_length
        self.model = self._build_model()
        self.class_weights = None

    def _build_model(self):
        inputs = layers.Input(shape=(self.seq_length, self.input_dim))
        x = layers.Conv1D(64, 3, activation='relu', padding='same', kernel_regularizer=regularizers.l2(1e-5))(inputs)
        x = layers.BatchNormalization()(x)
        x = layers.MaxPooling1D(2)(x)
        x = layers.Conv1D(128, 3, activation='relu', padding='same', kernel_regularizer=regularizers.l2(1e-5))(x)
        x = layers.BatchNormalization()(x)
        x = layers.MaxPooling1D(2)(x)
        shortcut = layers.Conv1D(128, 1)(inputs)
        shortcut = layers.BatchNormalization()(shortcut)
        shortcut = layers.MaxPooling1D(4)(shortcut)
        x = layers.add([x, shortcut])
        x = layers.Activation('relu')(x)
        x = layers.Bidirectional(layers.LSTM(64, return_sequences=True, kernel_regularizer=regularizers.l2(1e-5)))(x)
        x = layers.Dropout(0.4)(x)
        x = layers.Bidirectional(layers.LSTM(32, kernel_regularizer=regularizers.l2(1e-5)))(x)
        context_vector = x
        dense1 = layers.Dense(64, activation='relu', kernel_regularizer=regularizers.l2(1e-5))(context_vector)
        dense1 = layers.BatchNormalization()(dense1)
        dense1 = layers.Dropout(0.4)(dense1)
        logits = layers.Dense(self.num_classes)(dense1)
        temperature = 1.5
        scaled_logits = layers.Lambda(lambda x: x / temperature)(logits)
        outputs = layers.Activation('softmax')(scaled_logits)
        model = keras.Model(inputs=inputs, outputs=outputs)
        model.compile(optimizer=keras.optimizers.Adam(1e-3), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
        return model

    def compute_class_weights(self, y_train):
        y_train_int = y_train.astype(int)
        unique_classes = np.unique(y_train_int)
        class_counts = np.bincount(y_train_int)
        total = len(y_train_int)
        self.class_weights = {i: total / (len(unique_classes) * count) for i, count in enumerate(class_counts)}
        print("Class weights:", self.class_weights)
        return self.class_weights

    def train(self, X_train, y_train, X_val=None, y_val=None, epochs=50, batch_size=64):
        if self.class_weights is None:
            self.compute_class_weights(y_train)
        callbacks = [
            keras.callbacks.EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True),
            keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=7, min_lr=1e-7),
            keras.callbacks.ModelCheckpoint('best_layer2_model.h5', save_best_only=True, monitor='val_accuracy', mode='max'),
            keras.callbacks.TensorBoard(log_dir=f'./logs/layer2_{time.strftime("%Y%m%d-%H%M%S")}', histogram_freq=1)
        ]
        if X_val is not None and y_val is not None:
            history = self.model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size,
                                     validation_data=(X_val, y_val), callbacks=callbacks,
                                     class_weight=self.class_weights)
        else:
            history = self.model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size,
                                     validation_split=0.2, callbacks=callbacks, class_weight=self.class_weights)
        plt.figure(figsize=(12, 8))
        plt.subplot(2, 2, 1)
        plt.plot(history.history['loss'], label='Training Loss')
        plt.plot(history.history['val_loss'], label='Validation Loss')
        plt.title('Model Loss')
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.legend()
        plt.grid(True, alpha=0.3)
        plt.subplot(2, 2, 2)
        plt.plot(history.history['accuracy'], label='Training Accuracy')
        plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
        plt.title('Model Accuracy')
        plt.xlabel('Epoch')
        plt.ylabel('Accuracy')
        plt.legend()
        plt.grid(True, alpha=0.3)
        if 'lr' in history.history:
            plt.subplot(2, 2, 3)
            plt.plot(history.history['lr'], label='Learning Rate')
            plt.title('Learning Rate')
            plt.xlabel('Epoch')
            plt.ylabel('Learning Rate')
            plt.yscale('log')
            plt.grid(True, alpha=0.3)
        plt.tight_layout()
        plt.savefig('layer2_training_metrics.png')
        plt.close()
        return history

    def evaluate(self, X_test, y_test):
        test_loss, test_accuracy = self.model.evaluate(X_test, y_test, verbose=1)
        print(f"Test accuracy: {test_accuracy:.4f}")
        print(f"Test loss: {test_loss:.4f}")
        y_pred_probs = self.model.predict(X_test)
        y_pred = np.argmax(y_pred_probs, axis=1)
        report = classification_report(y_test, y_pred, output_dict=True)
        print("Classification Report:")
        print(classification_report(y_test, y_pred))
        cm = confusion_matrix(y_test, y_pred)
        plt.figure(figsize=(10, 8))
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                    xticklabels=np.unique(y_test), yticklabels=np.unique(y_test))
        plt.title('Confusion Matrix')
        plt.xlabel('Predicted')
        plt.ylabel('True')
        plt.savefig('confusion_matrix.png')
        plt.close()
        return report, y_pred, y_pred_probs

    def save_model(self, filepath):
        self.model.save(filepath)
        print(f"Model saved to {filepath}")

    def load_model(self, filepath):
        self.model = keras.models.load_model(filepath)
        print(f"Model loaded from {filepath}")
        return self.model
def parallel_scaling(X_chunk, scaler):
        return scaler.transform(X_chunk)

# --- Improved Feature Engineering and Data Processing ---
def preprocess_data(file_path, test_size=0.2, random_state=42, use_adasyn=True, top_k_features=50):
    """
    Enhanced preprocessing with dynamic scaling, feature selection, and robust outlier handling.
    """
    try:
        df = pd.read_csv(file_path)
        print(f"Successfully loaded dataset with {df.shape[0]} rows and {df.shape[1]} columns")
    except Exception as e:
        print(f"Error loading dataset: {e}")
        raise
    
    if df.empty or 'Attack_label' not in df.columns:
        raise ValueError("Dataset is empty or missing 'Attack_label' column.")
    
    X = df.drop(['Attack_label'], axis=1)
    y = df['Attack_label'].astype(int)
    
    print("Class distribution before preprocessing:")
    print(y.value_counts(normalize=True) * 100)
    
    class_counts = y.value_counts()
    if class_counts.min() / class_counts.max() < 0.01:
        print("Warning: Extreme class imbalance detected. Consider adjusting resampling strategy.")
    
    selector = SelectKBest(score_func=mutual_info_classif, k=min(top_k_features, X.shape[1]))
    X_selected = selector.fit_transform(X, y)
    selected_features = X.columns[selector.get_support()].tolist()
    print(f"Selected {len(selected_features)} features: {selected_features[:5]}...")
    joblib.dump(selector, 'feature_selector.pkl')
    
    if np.any(np.abs(stats.skew(X_selected)) > 1):
        print("High skewness detected, using PowerTransformer.")
        scaler = PowerTransformer(method='yeo-johnson')
        scaler_filename = 'power_transformer.pkl'
    elif np.any(X_selected < 0):
        print("Negative values detected, using QuantileTransformer.")
        scaler = QuantileTransformer(output_distribution='normal')
        scaler_filename = 'quantile_transformer.pkl'
    else:
        print("Using MaxAbsScaler for non-negative data.")
        scaler = MaxAbsScaler()
        scaler_filename = 'maxabs_scaler.pkl'
    
    # Replace multiprocessing with direct scaling
    scaler.fit(X_selected)
    X_scaled = scaler.transform(X_selected)
    
    joblib.dump(scaler, scaler_filename)
    print(f"Saved scaler as {scaler_filename}")
    
    # Use multiple CPU cores for LOF but avoid nested multiprocessing
    n_cores = min(mp.cpu_count(), 4)
    lof = LocalOutlierFactor(n_neighbors=20, contamination=0.05, n_jobs=n_cores)
    outlier_labels = lof.fit_predict(X_scaled)
    outlier_mask = outlier_labels == 1
    X_scaled = X_scaled[outlier_mask]
    y = y[outlier_mask]
    print(f"Removed {np.sum(~outlier_mask)} outliers using Local Outlier Factor.")
    
    X_train, X_test, y_train, y_test = train_test_split(
        X_scaled, y, test_size=test_size, random_state=random_state,
        stratify=y if len(np.unique(y)) > 1 else None
    )
    
    if use_adasyn and len(np.unique(y_train)) > 1:
        print("Applying ADASYN to balance classes...")
        adasyn = ADASYN(random_state=random_state)
        X_train, y_train = adasyn.fit_resample(X_train, y_train)
        print("Class distribution after ADASYN:")
        print(pd.Series(y_train).value_counts(normalize=True) * 100)
    
    return X_train, X_test, y_train, y_test, scaler

def create_sequences(data, labels, seq_length=10, stride=2, min_seq_variance=0.01):
    """
    Create sequences with dynamic length adjustment based on variance.
    """
    if len(data) < seq_length:
        raise ValueError(f"Data length ({len(data)}) is less than sequence length ({seq_length}).")
    
    data_variance = np.var(data, axis=0)
    effective_seq_length = seq_length
    if np.mean(data_variance) < min_seq_variance:
        effective_seq_length = max(5, seq_length // 2)
        print(f"Low variance detected, adjusting sequence length to {effective_seq_length}.")
    
    sequences, seq_labels = [], []
    labels = np.array(labels)
    
    n_sequences = (len(data) - effective_seq_length) // stride + 1
    if n_sequences <= 0:
        print("Warning: No sequences generated. Using full data as a single sequence.")
        return np.array([data]), np.array([labels[-1]])
    
    for i in range(0, len(data) - effective_seq_length + 1, stride):
        seq = data[i:i + effective_seq_length]
        sequences.append(seq)
        seq_labels.append(labels[i + effective_seq_length - 1])
    
    sequences = np.array(sequences)
    seq_labels = np.array(seq_labels)
    
    if sequences.size == 0:
        raise ValueError("No valid sequences created. Adjust seq_length or stride.")
    
    print(f"Created {len(sequences)} sequences with length {effective_seq_length}.")
    return sequences, seq_labels

# --- Main Execution ---
if __name__ == "__main__":
    # Create logs and model directories
    os.makedirs('./logs', exist_ok=True)
    os.makedirs('./models', exist_ok=True)
    
    # Set random seeds for reproducibility
    np.random.seed(42)
    tf.random.set_seed(42)
    
    # Start time is already defined in the notebook
    
    # Dataset path (update this path as needed)
    # Dataset path is already defined in the notebook
    # Preprocess data with improved feature engineering
    X_train, X_test, y_train, y_test, scaler = preprocess_data(
        dataset_path, test_size=0.2, use_adasyn=True, top_k_features=50
    )
    
    # Layer 1: Autoencoder for anomaly detection
    print("\nTraining Layer 1: Autoencoder...")
    layer1 = EnhancedAdaptiveNIDS(input_dim=X_train.shape[1])
    layer1.train(X_train, X_test, epochs=50, batch_size=64)
    
    # Detect anomalies using trained autoencoder
    anomalies, anomaly_indices, errors, confidence = layer1.detect_anomalies(X_test)
    
    print(f"Detected {len(anomaly_indices)} anomalies out of {len(X_test)} test samples.")
    print(f"Anomaly detection rate: {len(anomaly_indices)/len(X_test)*100:.2f}%")
    
    # Get encoded features for anomalies
    encoded_features = layer1.get_encoded_features(anomalies)
    
    # Handle case where no anomalies are detected
    if len(anomalies) == 0:
        print("No anomalies detected. Using top 10% of highest error samples.")
        top_n = int(len(X_test) * 0.1)
        sorted_indices = np.argsort(errors)[-top_n:]
        anomalies = X_test[sorted_indices]
        anomaly_indices = sorted_indices
        encoded_features = layer1.get_encoded_features(anomalies)
    
    # Get original labels for anomalies
    if isinstance(y_test, pd.Series):
        y_anomalies = y_test.iloc[anomaly_indices]
    else:
        y_anomalies = y_test[anomaly_indices]
    
    # Create sequences for Layer 2 with improved sequence creation
    X_layer2, y_layer2 = create_sequences(encoded_features, y_anomalies, seq_length=10, stride=2)
    
    # Split data for Layer 2
    if len(np.unique(y_layer2)) > 1:
        X_train_l2, X_test_l2, y_train_l2, y_test_l2 = train_test_split(
            X_layer2, y_layer2, test_size=0.2, random_state=42, stratify=y_layer2
        )
    else:
        X_train_l2, X_test_l2, y_train_l2, y_test_l2 = train_test_split(
            X_layer2, y_layer2, test_size=0.2, random_state=42
        )
    
    # Layer 2: CNN-BiLSTM Classification
    print("\nTraining Layer 2: CNN-BiLSTM...")
    layer2 = AdaptiveNIDSLayer2(
        input_dim=X_train_l2.shape[2],
        num_classes=len(np.unique(y_train_l2)),
        seq_length=10
    )
    layer2.train(X_train_l2, y_train_l2, X_test_l2, y_test_l2, epochs=50, batch_size=32)
    
    # Evaluate Layer 2
    class_names = [f"Class {i}" for i in range(len(np.unique(y_train_l2)))]
    report, y_pred, y_pred_probs = layer2.evaluate(X_test_l2, y_test_l2)
    
    # Save models with timestamp
    timestamp = time.strftime("%Y%m%d-%H%M%S")
    layer1.model.save(f'models/autoencoder_model_{timestamp}.h5')
    layer2.model.save(f'models/cnn_bilstm_model_{timestamp}.h5')
    
    # Save model architecture as image (optional)
    try:
        tf.keras.utils.plot_model(layer1.model, to_file='models/autoencoder_architecture.png',
                                  show_shapes=True, show_layer_names=True)
        tf.keras.utils.plot_model(layer2.model, to_file='models/cnn_bilstm_architecture.png',
                                  show_shapes=True, show_layer_names=True)
    except ImportError:
        print("Couldn't save model architecture images. Install pydot and graphviz.")
    
    # Save training configuration
    config = {
        'dataset_path': dataset_path,
        'preprocessing': {
            'use_adasyn': True,
            'top_k_features': 50,
            'test_size': 0.2
        },
        'layer1': {
            'latent_dim': layer1.latent_dim,
            'learning_rate': layer1.learning_rate,
            'threshold': layer1.threshold
        },
        'layer2': {
            'seq_length': layer2.seq_length,
            'class_weights': layer2.class_weights
        },
        'timestamp': timestamp
    }
    
    with open(f'models/training_config_{timestamp}.json', 'w') as f:
        json.dump(config, f, indent=4)
    
    elapsed_time = time.time() - start_time
    print(f"\nTotal execution time: {elapsed_time:.2f} seconds ({elapsed_time/60:.2f} minutes)")
    print(f"Models saved with timestamp: {timestamp}")

Successfully loaded dataset with 71401 rows and 45 columns
Class distribution before preprocessing:
Attack_label
1    85.994594
0    14.005406
Name: proportion, dtype: float64
Selected 44 features: ['Unnamed: 0', 'arp.opcode', 'arp.hw.size', 'icmp.checksum', 'icmp.seq_le']...
High skewness detected, using PowerTransformer.
Saved scaler as power_transformer.pkl
Removed 3570 outliers using Local Outlier Factor.
Applying ADASYN to balance classes...
Class distribution after ADASYN:
Attack_label
0    50.029598
1    49.970402
Name: proportion, dtype: float64

Training Layer 1: Autoencoder...
Epoch 1/50
1452/1452 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - loss: 1.4020 - val_loss: 0.3258 - learning_rate: 1.0000e-04
Epoch 2/50
1452/1452 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - loss: 0.4458 - val_loss: 0.6394 - learning_rate: 1.0000e-04
Epoch 3/50
1452/1452 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - loss: 0.2857 - val_loss: 1.4902 - learning_rate: 1.0000e-04
Epoch 4/50
1452/1452 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - lo

15/15 ━━━━━━━━━━━━━━━━━━━━ 4s 44ms/step - accuracy: 0.4665 - loss: 0.8767 - val_accuracy: 0.4779 - val_loss: 0.7015 - learning_rate: 0.0010
Epoch 2/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.5722 - loss: 0.7094 - val_accuracy: 0.4779 - val_loss: 0.7033 - learning_rate: 0.0010
Epoch 3/50
13/15 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.6666 - loss: 0.6235

15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 0.6631 - loss: 0.6252 - val_accuracy: 0.4867 - val_loss: 0.7030 - learning_rate: 0.0010
Epoch 4/50
13/15 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7282 - loss: 0.5730 

15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.7257 - loss: 0.5754 - val_accuracy: 0.5133 - val_loss: 0.6965 - learning_rate: 0.0010
Epoch 5/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.7592 - loss: 0.5086 - val_accuracy: 0.4956 - val_loss: 0.6932 - learning_rate: 0.0010
Epoch 6/50
13/15 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8463 - loss: 0.3499 

15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.8463 - loss: 0.3507 - val_accuracy: 0.6372 - val_loss: 0.6648 - learning_rate: 0.0010
Epoch 7/50
10/15 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.9335 - loss: 0.2412 

...

15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.9989 - loss: 0.0251 - val_accuracy: 0.8142 - val_loss: 0.4782 - learning_rate: 0.0010
Epoch 20/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.9969 - loss: 0.0215 - val_accuracy: 0.7965 - val_loss: 0.4982 - learning_rate: 0.0010
Epoch 21/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.9979 - loss: 0.0230 - val_accuracy: 0.8142 - val_loss: 0.4448 - learning_rate: 0.0010
Epoch 22/50
 9/15 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 1.0000 - loss: 0.0191 

15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.9982 - loss: 0.0242 - val_accuracy: 0.8407 - val_loss: 0.4723 - learning_rate: 0.0010
Epoch 23/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 1.0000 - loss: 0.0173 - val_accuracy: 0.8407 - val_loss: 0.4707 - learning_rate: 2.0000e-04
Epoch 24/50
 9/15 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 1.0000 - loss: 0.0149

15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 1.0000 - loss: 0.0150 - val_accuracy: 0.8584 - val_loss: 0.4582 - learning_rate: 2.0000e-04
Epoch 25/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 1.0000 - loss: 0.0198 - val_accuracy: 0.8584 - val_loss: 0.4490 - learning_rate: 2.0000e-04
Epoch 26/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 1.0000 - loss: 0.0148 - val_accuracy: 0.8584 - val_loss: 0.4513 - learning_rate: 2.0000e-04
Epoch 27/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 1.0000 - loss: 0.0131 - val_accuracy: 0.8584 - val_loss: 0.4588 - learning_rate: 2.0000e-04
Epoch 28/50
 9/15 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 1.0000 - loss: 0.0135 

15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 1.0000 - loss: 0.0143 - val_accuracy: 0.8673 - val_loss: 0.4715 - learning_rate: 2.0000e-04
Epoch 29/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 1.0000 - loss: 0.0137 - val_accuracy: 0.8673 - val_loss: 0.4781 - learning_rate: 2.0000e-04
Epoch 30/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 1.0000 - loss: 0.0128 - val_accuracy: 0.8584 - val_loss: 0.4822 - learning_rate: 4.0000e-05
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8036 - loss: 0.4054 
Test accuracy: 0.7876
Test loss: 0.4215
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 91ms/step


Classification Report:
              precision    recall  f1-score   support

           0       0.80      0.75      0.77        55
           1       0.77      0.83      0.80        58

    accuracy                           0.79       113
   macro avg       0.79      0.79      0.79       113
weighted avg       0.79      0.79      0.79       113

You must install pydot (`pip install pydot`) for `plot_model` to work.
You must install pydot (`pip install pydot`) for `plot_model` to work.

Total execution time: 263.75 seconds (4.40 minutes)
Models saved with timestamp: 20250319-132835
